<a href="https://colab.research.google.com/github/sasuraibito1125/google_colab/blob/main/Excel%E6%AF%94%E8%BC%83%E3%83%84%E3%83%BC%E3%83%AB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 概要

単純にExcelの内容をセル・バイ・セルで比較するツール。

# Excel比較関数定義

In [ ]:
!pip -qq install openpyxl

In [ ]:
# @title #### Excelファイル内容を正規化 { display-mode: "form" }

# @markdown 関数名：`normalize_excel_contents`

# @markdown キー項目：
# @markdown  * `sheet_name`
# @markdown  * `min_row`
# @markdown  * `max_row`
# @markdown  * `min_col`
# @markdown  * `max_col`
# @markdown  * `values`
def normalize_excel_contents(filename, verbose=False):
  '''Excelファイルの内容を正規化する。

    Parameters:
    -----------
      filename: 処理対象のExcelファイル名
      verbose : 冗長出力有効フラグ

    Returns:
    --------
      dict: 正規化されたデータ。
            キーは 'sheet_name', 'min_row', 'max_row', 'min_col', 'max_col', 'values'。
            'sheet_name'の型は文字列、'values'は各セルの型の二次元リスト、それ以外は正の整数。
  '''
  from openpyxl import load_workbook
  wb = load_workbook(filename, read_only=True)
  sheetnames = wb.sheetnames
  _result = []

  for name in sheetnames:
    normalized = {}
    normalized['sheet_name'] = name

    ws = wb[name]

    data_range = {
      'min_row': ws.min_row,
      'max_row': ws.max_row,
      'min_col': ws.min_column,
      'max_col': ws.max_column,
    }
    normalized['min_row'] = data_range['min_row']
    normalized['max_row'] = data_range['max_row']
    normalized['min_col'] = data_range['min_col']
    normalized['max_col'] = data_range['max_col']
    if verbose:
      print('行数:', data_range['max_row'] - data_range['min_row'],
            '列数:', data_range['max_col'] - data_range['min_col'])

    normalized['values'] = []
    for row in ws.iter_rows(**data_range):
      cells = []
      for cell in row:
        if verbose and cell.value:
          print(f'{cell.coordinate}({cell.row}, {cell.column})', cell.value)
        cells.append(cell.value)
      normalized['values'].append(cells)

    _result.append(normalized)

  return _result


In [ ]:
# @title #### 正規化されたExcelデータの差分を出力 { display-mode: "form" }

# @markdown 関数名：`diff_normalized_excel_data`

# @markdown キー項目：
# @markdown  * シート数
# @markdown  * シート
# @markdown   * シート名
# @markdown   * 最小行数
# @markdown   * 最大行数
# @markdown   * 最小列数
# @markdown   * 最大列数
# @markdown   * セル値
def diff_normalized_excel_data(first, second, verbose=False):
  '''正規化されたExcelデータの差分を出力する。

    Parameters:
    -----------
      first  : 一つ目の正規化されたExcelデータ
      second : 二つ目の正規化されたExcelデータ
      verbose: 冗長出力有効フラグ

    Returns:
    --------
      dict: 比較結果。
            キーは差分があったものだけが設定される。
            キーは 'シート数', 'シート名', '最小行数', '最大行数', '最小列数', '最大列数', and 'シート'。
            'シート'の値は'セル値'がキーで差分情報のdictの配列。
            セルの差分情報はキーが行番号と列番号のtuple，値が一番目と二番目のセル値のtuple。
  '''
  from itertools import zip_longest, count

  _result = {}

  len_1st = len(first)
  len_2nd = len(second)
  if len_1st != len_2nd:
    _result['シート数'] = (len_1st, len_2nd)
  if verbose:
    print('シート数が異なる' if 'シート数' in _result else 'シート数が同じ')

  def diff_ws(i, s1, s2, key, _result, result_key, verbose):
    '''シート情報の比較

    Parameters
    ----------
      i         : 比較対象のシートのインデックス
      s1        : 一つ目の比較対象のシートデータ
      s2        : 二つ目の比較対象のシートデータ
      key       : シートデータのキー名
      _result   : 比較結果出力用dict
      result_key: 比較結果出力用dictのキー名
      verbose   : 冗長出力有効フラグ

    Returns
    -------
      None
    '''
    if s1 and s2 and s1[key] != s2[key] or not s1 or not s2:
      diff = { i: (s1[key] if s1 else None, s2[key] if s2 else None) }

      if verbose:
        if not result_key in _result:
          print(f'{result_key}が異なる')
        print(result_key, diff)

      if result_key in _result:
        _result[result_key].append(diff)
      else:
        _result[result_key] = [diff]

  for i, s1, s2 in zip_longest(range(max(len_1st, len_2nd)), first, second):
    sheet_result = {}
    diff_ws(i, s1, s2, 'sheet_name', sheet_result, 'シート名', verbose)
    diff_ws(i, s1, s2, 'min_row', sheet_result, '最小行数', verbose)
    diff_ws(i, s1, s2, 'max_row', sheet_result, '最大行数', verbose)
    diff_ws(i, s1, s2, 'min_col', sheet_result, '最小列数', verbose)
    diff_ws(i, s1, s2, 'max_col', sheet_result, '最大列数', verbose)

    if verbose:
      for key in ['シート名', '最小行数', '最大行数', '最小列数', '最大列数']:
        if not key in sheet_result:
          print(f'{key}が同じ')

    # セル値チェックは両方ともあるものだけ
    if s1 and s2:
      for j, (r1, r2) in enumerate(zip(s1['values'], s2['values'])):
        for k, (c1, c2) in enumerate(zip(r1, r2)):
          if c1 != c2:
            diff = { (j, k): (c1, c2) }
            if 'セル値' in sheet_result:
              sheet_result['セル値'].append(diff)
            else:
              sheet_result['セル値'] = [diff]

    if 'シート' in _result:
      _result['シート'].append(sheet_result)
    else:
      _result['シート'] = [sheet_result]

  return _result


In [ ]:
# @title #### サマリ { display-mode: "form" }

# @markdown 関数名：`summarize`
def summarize(diff):
  '''サマリ情報をプリントする。

    Parameters
    ----------
      diff: 差分情報

    Returns
    -------
      None
  '''
  if not diff:
    print('比較失敗')
    return
  print('シートの差分数:', len(diff['シート']))
  for i, s in enumerate(diff['シート']):
    print(f'シート#{i}のセル値の差分数:', len(s['セル値']) if 'セル値' in s else None)

In [ ]:
# @title #### ディレクトリ内の全Excelファイルに対する比較 { display-mode: "form" }

# @markdown 関数名：`diff_in_dirs`

# @markdown 制約条件として比較するExcelのファイル名は同じものとする。また，一つ目のディレクトリ内のファイルとの比較のため，二つ目のディレクトリ内にのみ存在するファイルは比較されない（無視される）。
def diff_in_dirs(dir_1st = 'first', dir_2nd = 'second', summary=False, verbose=False):
  '''ディレクトリ内の全Excelファイルに対する比較する

    一つ目のディレクトリ内のExcelファイルと同名の二つ目のディレクトリ内のファイルを比較する。
    よって，二つ目のディレクトリにのみ存在するファイルは差分に反映されない。

    Parameters
    ----------
      dir_1st: 一つ目のディレクトリ名。デフォルトは'first'。
      dir_2nd: 二つ目のディレクトリ名。デフォルトは'second'。
      summary: サマリ出力有効フラグ。
      verbose: 冗長出力有効フラグ。

    Returns
    -------
      dict: 差分情報。
            キーはファイル名。

  '''
  import os

  diff_map = {}

  for filename in [f for f in os.listdir(dir_1st) if os.path.isfile(f'{dir_1st}/{f}') and f.endswith('.xlsx')]:
    try:
      diff_map[filename] = diff_normalized_excel_data(normalize_excel_contents(f'{dir_1st}/{filename}', verbose),
                                                      normalize_excel_contents(f'{dir_2nd}/{filename}', verbose),
                                                      verbose)
    except FileNotFoundError as e:
      print('ERROR: 対応するファイルがありません', e)
      diff_map[filename] = None
    except Exception as e:
      print('ERROR: 問題が発生しました', e)
      diff_map[filename] = None

    if verbose:
      print(f'{filename}:', diff_map[filename])

    if summary:
      print(f'SUMMARY({filename}):')
      summarize(diff_map[filename])

  return diff_map


# ツール利用

## 個別

### 手順

1. 比較するExcelファイルをアップロードする
2. 下記フォームにファイル名を設定する
3. 「比較ツール」のセクションを実行する
4. 「個別Excelファイル比較」を実行する
4. 「ディレクトリ内の全Excelファイルに対する比較」を実行する

In [ ]:
# @title #### 個別Excelファイルの比較 { display-mode: "form" }
# @markdown Excelファイル名（1つ目）
filename_1st = "foo.xlsx" # @param {type:"string"}
# @markdown Excelファイル名（2つ目）
filename_2nd = "bar.xlsx" # @param {type:"string"}
# @markdown サマリ出力
with_summrize = True # @param {type:"boolean"}
# @markdown デバッグ出力
verbose = False # @param {type:"boolean"}

result = diff_normalized_excel_data(normalize_excel_contents(filename_1st),
                                    normalize_excel_contents(filename_2nd),
                                    verbose)

if with_summrize:
  summarize(result)


## ディレクトリ比較

### 手順

1. 比較する2つのディレクトリを作成する
2. 各ディレクトリに比較するExcelファイルを**同じ名前**でアップロードする
3. 下記フォームにファイル名を設定する
4. 「比較ツール」のセクションを実行する
5. 「ディレクトリ内の全Excelファイルに対する比較」を実行する

In [ ]:
# @title #### ディレクトリ内の全Excelファイルの比較
# @markdown ディレクトリ名（1つ目）
dir_1st = "before" # @param {type:"string"}
# @markdown ディレクトリ名（2つ目）
dir_2nd = "after" # @param {type:"string"}
# @markdown サマリ出力
with_summrize = True # @param {type:"boolean"}
# @markdown デバッグ出力
verbose = False # @param {type:"boolean"}

result = diff_in_dirs(dir_1st, dir_2nd, with_summrize, verbose)
